In [2]:
# Import cell
import pathlib
from collections import defaultdict
import json
import pandas as pd
import yfinance as yf
import matplotlib.pyplot as plt


### take all column from ´company´ to ´similarity_score´ as variables, regress targetting returns

In [14]:
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import OneHotEncoder

# Load the merged data (assume it's saved as 'merged_data.csv')
df = pd.read_csv("trading_strat_data/merged_data.csv")

# Keep only the relevant columns (including Company to later map to Sector)
columns_to_keep = [
    "Year", "Company", "sentiment_score_positive", "sentiment_score_negative",
    "sentiment_score_polarity", "sentiment_score_subjectivity", "similarity_score",
    "Yearly_Return"
]
df = df[columns_to_keep]

# Define the sector mapping dictionary and add the Sector column
sector_mapping = {
    "AAPL": "Information Technology",
    "AMGN": "Health Care",
    "AMZN": "Consumer Discretionary",
    "AXP": "Financials",
    "BA": "Industrials",
    "CAT": "Industrials",
    "CRM": "Information Technology",
    "CSCO": "Information Technology",
    "CVX": "Energy",
    "DIS": "Communication Services",
    "GS": "Financials",
    "HD": "Consumer Discretionary",
    "HON": "Industrials",
    "IBM": "Information Technology",
    "JNJ": "Health Care",
    "JPM": "Financials",
    "KO": "Consumer Staples",
    "MCD": "Consumer Discretionary",
    "MMM": "Industrials",
    "MRK": "Health Care",
    "MSFT": "Information Technology",
    "NKE": "Consumer Discretionary",
    "NVDA": "Information Technology",
    "PG": "Consumer Staples",
    "SHW": "Materials",
    "TRV": "Financials",
    "UNH": "Health Care",
    "V": "Financials",
    "VZ": "Communication Services",
    "WMT": "Consumer Staples"
}
df["Sector"] = df["Company"].map(sector_mapping)

# One-hot encode the 'Company' column
encoder_company = OneHotEncoder(sparse_output=False, drop='first')
company_encoded = encoder_company.fit_transform(df[['Company']])
company_encoded_df = pd.DataFrame(company_encoded, columns=encoder_company.get_feature_names_out(['Company']))

# One-hot encode the 'Sector' column
encoder_sector = OneHotEncoder(sparse_output=False, drop='first')
sector_encoded = encoder_sector.fit_transform(df[['Sector']])
sector_encoded_df = pd.DataFrame(sector_encoded, columns=encoder_sector.get_feature_names_out(['Sector']))

# Concatenate the encoded columns with the original dataframe and drop the original Company and Sector columns
df = pd.concat([df.reset_index(drop=True), company_encoded_df.reset_index(drop=True), sector_encoded_df.reset_index(drop=True)], axis=1)
df.drop(columns=["Company", "Sector"], inplace=True)

# Define the target and features
target = "Yearly_Return"
features = [
    "sentiment_score_positive", "sentiment_score_negative",
    "sentiment_score_polarity", "sentiment_score_subjectivity", "similarity_score"
] + list(company_encoded_df.columns) + list(sector_encoded_df.columns)

# Sort data by Year to maintain time order
df.sort_values("Year", inplace=True)

# Perform a time-based split: training on 2011-2020, testing on 2021-2024
train_data = df[df["Year"] <= 2020]
test_data  = df[df["Year"] > 2020]

X_train = train_data[features]
y_train = train_data[target]
X_test  = test_data[features]
y_test  = test_data[target]

# Initialize and train the linear regression model
lr_model = LinearRegression()
lr_model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = lr_model.predict(X_test)

# Evaluate the model using Mean Squared Error (MSE)
mse = mean_squared_error(y_test, y_pred)
print("Mean Squared Error on test set:", mse)


Mean Squared Error on test set: 0.13383163051682934


## batched by company

## I) 

-try to pool all together/ not pool and keep company per company
(training assuming that all synthax of files "10_K_info.txt" are in similar style: hence we don't trian company per company?)

-try algo reg, NeuralNet, etc?

-which inputs

## II)
Trading strat and backtest?